<a href="https://colab.research.google.com/github/catastropiyush/2022_simons_collab_pyscf_workshop/blob/main/demos/04_Energy_vs_Lattice_Constant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jamesETsmith/2022_simons_collab_pyscf_workshop/blob/main/demos/04_Energy_vs_Lattice_Constant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Jupyter notebook

* We need to install a few things before we get started
  * [PySCF](https://pyscf.org/) for the quantum chemsitry
  * [NumPy](https://numpy.org/) for manipulating arrays
  * [pandas](https://pandas.pydata.org/) for manipulating table data
  * [plotly](https://plotly.com/python/) for plotting
  * [SciPy](https://scipy.org/) for curve fitting

In [1]:
%pip install -q numpy pyscf plotly==5.8.0 scipy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 9.4 MB/s eta 0:00:00


# Structural Properties of Materials

Here, we investigate structural properties of solid Si, in particular the equilibrium lattice parameter, bulk modulus and cohesive energy.

We run our calculations at various lattice parameters. Then we plot the energy vs volume per atom curve and fit the Birch-Murnaghan equation which gives us these properties, e.g. see https://en.wikipedia.org/wiki/Birch%E2%80%93Murnaghan_equation_of_state .

In [2]:
import numpy as np
import pandas as pd
import plotly.express
import plotly.graph_objects
from scipy.optimize import curve_fit
from pyscf.pbc import gto, scf # note the pyscf.pbc for solid calculations

## Calculations of FCC Si at various lattice parameters
We use the calculation set-up from the last notebook, using FCC Si in a gth-dzvp basis, at a 333 **k** point mesh, with DFT LDA. Note that this is not enough for production calculations as the basis set, finite size and methods errors have not eliminated here.

In [3]:
def get_fcc_si_cell(latt_param):
    # Setting up primitive face centered cubic (FCC) cell
    cell_lattice = 0.5*latt_param*np.asarray([[1.0, 0.0, 1.0],
                                              [1.0, 1.0, 0.0],
                                              [0.0, 1.0, 1.0]])
    qlp = latt_param*0.25
    cell_xyz = f"""Si        0.00000    0.00000   0.00000
                   Si        {qlp}      {qlp}     {qlp}"""
    cell = gto.Cell(a=cell_lattice, atom=cell_xyz, basis="gth-dzvp", pseudo="gth-pade", verbose=2)
    cell.build()
    return cell

In [4]:
lda_es = []
k = 3  # Using a 333 k point mesh.
n = 9  # 9 data points
latt_params = [np.round(5.56 - (n//2)*0.01 + 0.01*i,2) for i in range(n)]
for latt_param in latt_params:
    cell = get_fcc_si_cell(latt_param)
    mykmf = scf.KRKS(cell, cell.make_kpts([k,k,k]), xc="lda").run()
    lda_es.append(mykmf.e_tot)
    print(latt_param, mykmf.e_tot)
print("done!")

WARNING!
  Very diffused basis functions are found in the basis set. They may lead to severe
  linear dependence and numerical instability.  You can set  cell.exp_to_discard=0.1
  to remove the diffused Gaussians whose exponents are less than 0.1.



5.52 -7.5267894472187


WARNING!
  Very diffused basis functions are found in the basis set. They may lead to severe
  linear dependence and numerical instability.  You can set  cell.exp_to_discard=0.1
  to remove the diffused Gaussians whose exponents are less than 0.1.



5.53 -7.526870647172146


WARNING!
  Very diffused basis functions are found in the basis set. They may lead to severe
  linear dependence and numerical instability.  You can set  cell.exp_to_discard=0.1
  to remove the diffused Gaussians whose exponents are less than 0.1.



...

5.58 -7.52692905489963


WARNING!
  Very diffused basis functions are found in the basis set. They may lead to severe
  linear dependence and numerical instability.  You can set  cell.exp_to_discard=0.1
  to remove the diffused Gaussians whose exponents are less than 0.1.



5.59 -7.526874051798782


WARNING!
  Very diffused basis functions are found in the basis set. They may lead to severe
  linear dependence and numerical instability.  You can set  cell.exp_to_discard=0.1
  to remove the diffused Gaussians whose exponents are less than 0.1.



5.6 -7.526797742296893
done!


## Plotting and Fitting

In [5]:
# Collect data
# Volume per Si atom. This is latt_params^3/4, as there are four Si atoms in a cubic unit cell of sides latt_param.
v_si = [latt_param**3/4.0 for latt_param in latt_params]
energies_si = np.asarray(lda_es)/2.0 # Energy per Si atom. There were two Si atoms in our primitive unit cell.
# Plotting
fig = plotly.express.line(x=v_si, y=energies_si, title="Binding Curve", markers=True)
fig.update_layout(xaxis_title="Volume per Si/Ang^3", yaxis_title="Energy per Si/ha")
fig.update_traces(marker_size=12)
#fig.update_xaxes(range=[0.0, 1.01])
fig.show() # It's interactive!

Looks like the minimum is close to our middle data point, at 5.56 ang. Now, we fit the Birch-Murnaghan equation.

In [6]:
def birch_murnaghan_fit(x, e, v, b, bdash):
    return (e + (9.0/16.0)*v*b*((((v/x)**(2.0/3.0) - 1.0)**3)*bdash +
                                (((v/x)**(2.0/3.0) - 1.0)**2)*(6.0 - 4.0*((v/x)**(2.0/3.0)))))

In [7]:
init_guess = [energies_si[n//2], v_si[n//2], 1, 1]
opt_params, _ = curve_fit(birch_murnaghan_fit, v_si, energies_si, p0=init_guess)
print("Min energy per Si", np.round(opt_params[0],5), "ha")
print("Equilibrium volume per Si", np.round(opt_params[1],5), "ang^3, which is at lattice parameter",
      np.round((4.0*opt_params[1])**(1.0/3.0),3), "ang.")
# See https://en.wikipedia.org/wiki/Hartree for unit convertion (ha to J, and then J/ang to Pa)
print("Bulk Modulus is", np.round(opt_params[2],5), "ha/ang^3, which is", np.round(opt_params[2]*4.3597447*10**3,2), "GPa")

Min energy per Si -3.76349 ha
Equilibrium volume per Si 42.96953 ang^3, which is at lattice parameter 5.56 ang.
Bulk Modulus is 0.00899 ha/ang^3, which is 39.19 GPa


The first parameter is the energy (which is not meaningful) at the minimum which is given as -3.76349 ha by the fit.
The equilibrium lattice parameter is at 5.56 ang, consistent with our previous observation.
The bulk modulus is about 39 GPa.

In [8]:
# Now let's plot the fit, too
v_ctd = [42 + 0.001*i for i in range(2000)]
fig2 = plotly.graph_objects.Figure()
fig2.add_trace(plotly.graph_objects.Scatter(x=v_si, y=energies_si, name="LDA Data", mode='markers'))
fig2.add_trace(plotly.graph_objects.Scatter(x=v_ctd, y=[birch_murnaghan_fit(v, *opt_params) for v in v_ctd], name="Fit", mode='lines'))
fig2.update_layout(xaxis_title="Volume per Si/Ang^3", yaxis_title="Energy per Si/ha")
fig2.update_traces(marker_size=12)
fig2.show() # It's interactive!

## Including single atom energies

To improve the equilibrium lattice parameter and bulk modulus estimates, and to estimate the cohesive energy, we need to evaluate the energy of a single Si atom.

The cohesive energy is evaluated as the energy difference of the energy an Si atom has in the solid and the energy it has surrounded by no other atom.

For the single atom energy, we remove the other atoms but keep their basis functions at the positions they were. We also use the same pseudopotential. This helps correct the basis set superposition error.

In [9]:
# This dictionary gives (not quite converged) single Si atom energies.
single_energies_si = {"latt_param": [5.52, 5.53, 5.54, 5.55, 5.56, 5.57, 5.58, 5.59, 5.6],
                      "E_LDA_s": [-3.622921557772764, -3.62292196154122, -3.622922391188982, -3.6229232993272307,
                                  -3.622924186113336, -3.6229249777570565, -3.6229255226694903, -3.622926871590588,
                                  -3.622926286317038]}
single_energies = pd.DataFrame(single_energies_si)
single_energies

,latt_param,E_LDA_s
0,5.52,-3.622922
1,5.53,-3.622922
2,5.54,-3.622922
3,5.55,-3.622923
4,5.56,-3.622924
5,5.57,-3.622925
6,5.58,-3.622926
7,5.59,-3.622927
8,5.60,-3.622926


In [10]:
# Converting previous bulk data into a DataFrame and merging with new DataFrame
bulk_energies = pd.DataFrame({"latt_param": latt_params, "E_LDA_b": energies_si})
bulk_energies = bulk_energies.merge(single_energies, on="latt_param") # this makes sure we are not misaligning lattice parameters
bulk_energies["E_LDA_diff"] = bulk_energies["E_LDA_b"] - bulk_energies["E_LDA_s"]
bulk_energies

,latt_param,E_LDA_b,E_LDA_s,E_LDA_diff
0,5.52,-3.763395,-3.622922,-0.140473
1,5.53,-3.763435,-3.622922,-0.140513
2,5.54,-3.763464,-3.622922,-0.140542
3,5.55,-3.763481,-3.622923,-0.140558
4,5.56,-3.763487,-3.622924,-0.140563
5,5.57,-3.763481,-3.622925,-0.140556
6,5.58,-3.763465,-3.622926,-0.140539
7,5.59,-3.763437,-3.622927,-0.140510
8,5.60,-3.763399,-3.622926,-0.140473


In [11]:
# Now fitting to "E_LDA_diff", whose minimum is the cohesive energy estimate.
init_guess = [bulk_energies["E_LDA_diff"].iloc[n//2], v_si[n//2], 1, 1]
opt_params, _ = curve_fit(birch_murnaghan_fit, v_si, bulk_energies["E_LDA_diff"], p0=init_guess)
print("Min energy per Si", np.round(opt_params[0],5), "ha")
print("Equilibrium volume per Si", np.round(opt_params[1],5), "ang^3, which is at lattice parameter",
      np.round((4.0*opt_params[1])**(1.0/3.0),3), "ang.")
# See https://en.wikipedia.org/wiki/Hartree for unit convertion (ha to J, and then J/ang to Pa)
print("Bulk Modulus is", np.round(opt_params[2],5), "ha/ang^3, which is", np.round(opt_params[2]*4.3597447*10**3,2), "GPa")

Min energy per Si -0.14056 ha
Equilibrium volume per Si 42.95036 ang^3, which is at lattice parameter 5.559 ang.
Bulk Modulus is 0.009 ha/ang^3, which is 39.22 GPa


The bulk modulus and equilibrium lattice parameter have not been affected by much (but might be in other cases). The cohesive energy estimate is -0.1406 ha.

Note that this was all at zero temperature. Temperature effects have been ignored.